In [1]:
# import the necessary packages

import cv2

In [5]:
imgPath = 'images/view1.jpeg'
blurry = 'blurry/image1.jpg'

imagePaths = [f'images/view{i}./images/view{i}.jpeg' for i in range(1,11)]+[blurry]

threshold = 100

### Using Laplacian operator to detect blurry image

In [27]:
def variance_of_laplacian(image):
    # compute the Laplacian of the image and then return the focus
    # measure, which is simply the variance of the Laplacian
    return cv2.Laplacian(image, cv2.CV_64F).var()


def detect_blurry(imagePath):
    # load the image, convert it to grayscale, and compute the
    # focus measure of the image using the Variance of Laplacian
    # method
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    fm = variance_of_laplacian(gray)
    
    text = 'Not Blurry'
    # if the focus measure is less than the supplied threshold,
    # then the image should be considered "blurry"
    if fm < threshold:
        text = 'Blurry'  
        
    # resize the image if too big    
    if image.shape[1]//4 > 400 or image.shape[0]//4 > 400 :        
        dim = (image.shape[1]//4, image.shape[0]//4)
        image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    
    cv2.putText(image, '{}: {:.2f}'.format(text, fm), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)
    # show the image <- comment out to turn off 
#     cv2.imshow('Image', image)
#     key = cv2.waitKey(0)
    
    # save output image
    filename = imagePath.split('.')[-2]
    filename = filename.split('/')[-1]
    filename = 'output/blurry/'+filename+'.jpeg'
    cv2.imwrite(filename, image)
  
    
    return text, fm

In [28]:
# loop over the input images
result = []
for ip in imagePaths:
    text, fm = detect_blurry(ip)
    result.append(ip + ' -> '+ text + ' ('+str(fm)+')')

In [26]:
print(*result, sep='\n')

images/view1./images/view1.jpeg -> Not Blurry (522.1364012364561)
images/view2./images/view2.jpeg -> Not Blurry (167.56627389868947)
images/view3./images/view3.jpeg -> Not Blurry (212.52306979666523)
images/view4./images/view4.jpeg -> Not Blurry (360.2202692149387)
images/view5./images/view5.jpeg -> Blurry (83.2438442684417)
images/view6./images/view6.jpeg -> Not Blurry (212.16054984382745)
images/view7./images/view7.jpeg -> Not Blurry (328.8413763372435)
images/view8./images/view8.jpeg -> Not Blurry (270.1843277052043)
images/view9./images/view9.jpeg -> Not Blurry (270.5144053417482)
images/view10./images/view10.jpeg -> Not Blurry (165.53637893923673)
blurry/image1.jpg -> Blurry (3.2903089202774845)


In [9]:
image = cv2.imread(imagePaths[0])
image.shape

(3024, 4032, 3)

### Code to generate mask images from json

In [3]:
import os
import cv2
import json
import numpy as np 

source_folder = os.path.join(os.getcwd(), "images")
json_path = "cars_json.json"                     # Relative to root directory
count = 0                                           # Count of total images saved
file_bbs = {}                                       # Dictionary containing polygon coordinates for mask
MASK_WIDTH = 4032                     # Dimensions should match those of ground truth image
MASK_HEIGHT = 3024

# Read JSON file
with open(json_path) as f:
    data = json.load(f)

# Extract X and Y coordinates if available and update dictionary
def add_to_dict(data, itr, key, count):
    try:
        x_points = data[itr]["regions"][count]["shape_attributes"]["all_points_x"]
        y_points = data[itr]["regions"][count]["shape_attributes"]["all_points_y"]
    except:
        print("No BB. Skipping", key)
        return
    
    all_points = []
    for i, x in enumerate(x_points):
        all_points.append([x, y_points[i]])
    
    file_bbs[key] = all_points
    

for itr in data:
    file_name_json = data[itr]["filename"]
    sub_count = 0               # Contains count of masks for a single ground truth image
    
    if len(data[itr]["regions"]) > 1:
        for _ in range(len(data[itr]["regions"])):
            key = file_name_json[:-4] + "*" + str(sub_count+1)
            add_to_dict(data, itr, key, sub_count)
            sub_count += 1
    else:
        add_to_dict(data, itr, file_name_json[:-4], 0)


print("\nDict size: ", len(file_bbs))

for file_name in os.listdir(source_folder):
    to_save_folder = os.path.join(source_folder, file_name[:-4])
    image_folder = os.path.join(to_save_folder, "images")
    mask_folder = os.path.join(to_save_folder, "masks")
    curr_img = os.path.join(source_folder, file_name)
    
    # make folders and copy image to new location
    os.mkdir(to_save_folder)
    os.mkdir(image_folder)
    os.mkdir(mask_folder)
    os.rename(curr_img, os.path.join(image_folder, file_name))
        
# For each entry in dictionary, generate mask and save in correponding 
# folder
for itr in file_bbs:
    num_masks = itr.split("*")
    to_save_folder = os.path.join(source_folder, num_masks[0])
    mask_folder = os.path.join(to_save_folder, "masks")
    mask = np.zeros((MASK_HEIGHT, MASK_WIDTH))
    try:
        arr = np.array(file_bbs[itr])
    except:
        print("Not found:", itr)
        continue
    count += 1
    cv2.fillPoly(mask, [arr], color=(255))
    
    if len(num_masks) > 1:
        cv2.imwrite(os.path.join(mask_folder, itr.replace("*", "_") + ".png") , mask)    
    else:
        cv2.imwrite(os.path.join(mask_folder, itr + ".png") , mask)
        
print("Images saved:", count)



Dict size:  10
Images saved: 10
